In [1]:
import pandas as pd
import os 
import glob
import re
import numpy as np

In [30]:
character = os.listdir('Data_results/')
character = ['Data_results/' + file for file in character]
character

['Data_results/Raju']

In [31]:
result_dict = {}
contexts_all = []
for character_file in character:
    files = glob.glob(character_file+'/*.csv')
    character = character_file.split('/')[1]
    contexts = [re.findall(r'(?<=_)[^_]*(?=\.[^.]+$)',file)[0].lower() for file in files]
    
    # 场景×情绪
    result_df = pd.DataFrame(index = contexts,
                    columns = ['anger','disgust', 'fear', 'happy', 'sad',
                      'surprised','neutral'])
    
    for file in files:
        # 提取场景
        context = re.findall(r'(?<=_)[^_]*(?=\.[^.]+$)',file)[0].lower()
        contexts_all.append(context)
        
        raw = pd.read_csv(file,index_col=0)
        raw = raw.dropna()
        table = raw.groupby('name')['emotion'].value_counts().unstack(fill_value=0)
        
        for emo in table.columns:
            result_df.loc[context,emo] = table.loc[character,emo]
    
    result_df = result_df.apply(lambda row: row/row.sum()*100, axis=1)
    result_df = result_df.fillna(0)
    result_dict[character] = result_df

In [32]:
result_df

,anger,disgust,fear,happy,sad,surprised,neutral
context1,18.181818,0.0,0.0,12.121212,66.666667,0.0,3.030303
context2,90.000000,0.0,0.0,0.000000,10.000000,0.0,0.000000
context3,0.000000,0.0,0.0,0.000000,0.000000,0.0,100.000000


In [33]:
with pd.ExcelWriter('Character_Context.xlsx') as writer:
    for child in result_dict.keys():
        result_dict[character].to_excel(writer, sheet_name = character)

In [34]:
set(contexts_all)

{'context1', 'context2', 'context3'}